In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the paths to the training and testing data
train_data_dir = 'data/training_data'
test_data_dir = 'data/testing_data'

# Define image dimensions and batch size
img_width, img_height = 64, 64
batch_size = 32

# Create data generators with augmentation for training and testing data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse'  # for multi-class classification
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse'
)


**MODEL ARCHITECTURE**

---

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(36, activation='softmax'))  # 36 classes (0-9 and A-Z)

model.summary()


In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
epochs = 30

model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size
)


In [ ]:
loss, accuracy = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print("Test Accuracy:", accuracy)


In [ ]:
model.save("character_classifier_model4.h5")

In [2]:
import cv2
import numpy as np
import tensorflow as tf

# Load the image
image = cv2.imread('images\Arthur.jpg')
model = tf.keras.models.load_model('F:\Projects\OCRProjectV5\character_classifier_model4.h5')

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply thresholding to binarize the image
_, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# Perform morphological operations to clean up the image
kernel = np.ones((3,3), np.uint8)
cleaned_image = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)

# Find contours in the cleaned image
contours, _ = cv2.findContours(cleaned_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Sort contours based on their x-coordinate
contours = sorted(contours, key=lambda c: cv2.boundingRect(c)[0])

# Initialize a string to store the recognized word
recognized_word = ''
padding = 12

# Iterate through each contour
for contour in contours:
    # Get the bounding box coordinates
    x, y, w, h = cv2.boundingRect(contour)
    # Add padding to expand the bounding box
    x -= padding
    y -= padding
    w += 2 * padding
    h += 2 * padding
  
    # Crop the character region from the original image
    character = image[y:y+h, x:x+w]
    
    # Check if the cropped character region is empty
    if character.size == 0:
        continue
    
    # Preprocess the character (resize to match model input shape and normalize)
    character = cv2.resize(character, (64, 64))  # Resize the character
    character = character.astype('float32') / 255.0  # Normalize pixel values
    character = np.expand_dims(character, axis=0)  # Add batch dimension
    
    # Perform character recognition
    predictions = model.predict(character)
    
    # Decode the predicted character
    mapping = '0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    predicted_character = mapping[np.argmax(predictions)]
    
    # Append the recognized character to the word
    recognized_word += predicted_character

    # Display the recognized character
    cv2.putText(image, predicted_character, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

# Print the recognized word
print("Recognized word:", recognized_word)

# Display the image with recognized characters
cv2.imshow('Recognized Characters', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Recognized word: ARTHUR
